In [79]:
%%capture
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score

In [80]:
QPATH = "Quantlet/5-domain-pre-training"

In [81]:
import sys

sys.path.append("../4-qode2desc")
IN_COLAB = "google.colab" in sys.modules

import os

if IN_COLAB:
    os.chdir(
        f"/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}"
    )
else:
    %load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [82]:
import pickle
import json
import re
import sys
from IPython.display import display
import datetime

from tqdm import tqdm
import pandas as pd
import numpy as np

tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import nltk

nltk.download("punkt")
import evaluate

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [83]:
import importlib
import analysis_modules

importlib.reload(analysis_modules)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'analysis_modules' from '/usr/net/zinovyee.hub/IRTG/MLSC/Encode-the-Qode/Quantlet/5-domain-pre-training/../4-qode2desc/analysis_modules.py'>

In [88]:
model_name = "CodeT5"

DATE = "20231021"

SAMPLE = "test"
if SAMPLE == "test":
    load_best_model_at_end = False
else:
    load_best_model_at_end = None

# tokenization
encoder_max_length = 300
decoder_max_length = 80
RS = 1
LR = 5e-5

EPOCHS = 4
TRAIN_BATCH = 4
EVAL_BATCH = 4

WARMUP_STEPS = 500
WEIGHT_DECAY = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 4
SAVE_STRATEGY = "steps"

LABEL_SMOOTHING = 0.1
PREDICT_GENERATE = True

EVAL_COLUMNS = [
    "eval_loss",
    "eval_rouge1",
    "eval_rouge2",
    "eval_rougeL",
    "eval_rougeLsum",
    "eval_bleu",
    "eval_gen_len",
]


analysis_name = model_name + "-" + str(EPOCHS)

analysis_name = analysis_name + "-" + DATE

print(analysis_name)

CodeT5-4-20231021


In [89]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [90]:
print(analysis_name)
analysis_modules.scs_analyze(
    analysis_name=analysis_name,
    model_name=model_name,
    train_data_path=f"../../data/preprocessed/ArXiv/",
    train_data_name=f"arxiv_pretrain_train.json",
    val_data_path=f"../../data/preprocessed/ArXiv/",
    val_data_name=f"arxiv_pretrain_test.json",
    encoder_max_length=encoder_max_length,
    decoder_max_length=decoder_max_length,
    random_state=RS,
    eval_columns_list=EVAL_COLUMNS,
    learning_rate=LR,
    epochs=EPOCHS,
    train_batch=TRAIN_BATCH,
    eval_batch=EVAL_BATCH,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    logging_stes=LOGGING_STEPS,
    save_total_lim=SAVE_TOTAL_LIM,
    save_strategy=SAVE_STRATEGY,
    label_smooting=LABEL_SMOOTHING,
    predict_generate=PREDICT_GENERATE,
    load_best_model_at_end=load_best_model_at_end,
    evaluation_strategy='no'
)

CodeT5-4-20231021
CodeT5-4-20231021
cuda
cuda


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/16855 [00:00<?, ? examples/s]

/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      5.419        0.173        0.054         0.15            0.15   

   eval_bleu  eval_gen_len  
0      0.004        18.456  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,5.093800
200,4.238600
300,3.981500
400,3.890500
500,3.876400
600,3.731400
700,3.633200
800,3.499900
900,3.554300
1000,3.512400



KeyboardInterrupt



In [ ]:
gc.collect()
torch.cuda.empty_cache()